In [50]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import requests
import json

In [30]:
def yahoo_opt_clean(x, type):
    x = pd.io.json.json_normalize(x['optionChain']['result'][0]['options'][0][type])
    x = x[['ask', 'bid', 'expiration', 'strike', 'inTheMoney']]
    if type == 'calls':
        x['type'] = 'C'
    elif type == 'puts':
        x['type'] = 'P'
    else:
        raise ValueError('Unknown option type')
    return x


def get_options():
    url = 'https://query2.finance.yahoo.com/v7/finance/options/SPY'
    content = requests.get(url).text
    content = json.loads(content)
    current_price = content['optionChain']['result'][0]['quote']['regularMarketPrice']
    current_date = content['optionChain']['result'][0]['quote']['regularMarketTime']
    dates = content['optionChain']['result'][0]['expirationDates']
    options = yahoo_opt_clean(content, 'calls')
    df = yahoo_opt_clean(content, 'puts')
    options = options.append(df, ignore_index=True)
    for i in range(1, 10):
        content = requests.get(url + '?date=' + str(dates[i])).text
        content = json.loads(content)
        df = yahoo_opt_clean(content, 'calls')
        options = options.append(df, ignore_index=True)
        df = yahoo_opt_clean(content, 'puts')
        options = options.append(df, ignore_index=True)
    return options, current_price, current_date

options, underlying, date = get_options()

In [31]:
options.drop(options[options.inTheMoney == True].index, inplace=True)
options['price'] = (options['ask'] - options['bid'])/2 + options['bid']
options.drop(['ask', 'bid', 'inTheMoney'], axis=1, inplace=True)
options.reset_index(drop=True, inplace=True)

In [35]:
print(options.head())
print(options.shape)

   expiration  strike type  price
0  1517961600   276.0    C  1.420
1  1517961600   277.0    C  0.980
2  1517961600   277.5    C  0.775
3  1517961600   278.0    C  0.630
4  1517961600   278.5    C  0.500
(711, 4)


In [70]:
df = options.copy(deep=True)
# Pivot
df = df.pivot(index='expiration', columns='strike', values='price')
# Only keep columns with no NaN's
df = df[df.columns[~df.isna().any()]]
df.head()

strike,250.0,255.0,260.0,265.0,270.0,275.0,280.0,281.0,282.0,283.0,284.0,285.0,286.0,287.0,288.0,289.0,290.0,295.0,300.0
expiration,,,,,,,,,,,,,,,,,,,
1517961600,0.085,0.120,0.175,0.305,0.64,1.710,0.250,0.155,0.110,0.075,0.045,0.030,0.025,0.015,0.015,0.015,0.010,0.005,0.005
1518134400,0.135,0.190,0.285,0.490,0.97,2.180,0.510,0.330,0.215,0.145,0.095,0.065,0.040,0.030,0.025,0.015,0.015,0.005,0.005
1518566400,0.265,0.370,0.550,0.875,1.49,2.780,0.885,0.635,0.440,0.310,0.210,0.150,0.110,0.080,0.060,0.050,0.040,0.020,0.010
1518739200,0.320,0.460,0.680,1.045,1.71,3.045,1.140,0.845,0.610,0.440,0.310,0.220,0.175,0.115,0.095,0.075,0.065,0.020,0.010
1519171200,0.395,0.565,0.825,1.245,1.97,3.310,1.355,1.030,0.760,0.555,0.395,0.285,0.205,0.150,0.125,0.085,0.065,0.030,0.025


In [153]:
def d1d2(S, K, r, sigma, T):
    # Takes T in years
    d1 = (np.log(S / K) + ((r + ((sigma**2)/2))*T)) / (sigma * np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    return d1, d2


def price_call(S, K, r, sigma, T):
    T /= 365 # Converts T from days to years
    d1, d2 = d1d2(S, K, r, sigma, T)
    c = (S * norm.cdf(d1)) - (K * np.exp(-1 * r * T) * norm.cdf(d2))
    return c

def price_put(S, K, r, sigma, T):
    T /= 365 # Converts T from days to years
    d1, d2 = d1d2(S, K, r, sigma, T)
    c = (K * np.exp(-1 * r * T) * norm.cdf(-d2)) - (S * norm.cdf(-d1))
    return c

def call_vega(S, K, r, sigma, T):
    T /= 365 # Converts T from days to years
    d1, d2 = d1d2(S, K, r, sigma, T)
    v = S*np.sqrt(T)*norm.pdf(d1)/100 # IDK why you have to divide here
    return v

S = 5
K = 4
r = 0.02
sigma = 0.2
T = 10

print(price_call(S, K, r, sigma, T))
print(call_vega(S, K, r, sigma, T))

1.0021911804438326
3.5912891234840177e-13
